In [55]:
import numpy as np
import pandas as pd

In [56]:
df = pd.read_csv('Reddit_Data.csv')
df1 =df.copy()
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [57]:
df.shape

(37249, 2)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [59]:
df.isnull().sum()

clean_comment    100
category           0
dtype: int64

In [60]:
df.dropna(inplace=True)

In [61]:
df.shape

(37149, 2)

In [62]:
df.duplicated().sum()

350

In [63]:
df.drop_duplicates(inplace=True)

In [64]:
df[(df['clean_comment'].str.strip() == '')]

,clean_comment,category
181,,0
4432,\n,0
10592,,0
16173,,0
32149,\n,0
34959,,0


In [65]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [66]:
# Convert the 'clean_comment' column to lowercase
df['clean_comment'] = df['clean_comment'].str.lower()

# Verify the transformation by displaying the first few rows
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [67]:
# Remove trailing and leading whitespaces from the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].str.strip()

# Verify the transformation by checking for any remaining trailing whitespaces
df['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' ')).sum()

0

In [68]:
# Identify comments containing URLs
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
comments_with_urls = df[df['clean_comment'].str.contains(url_pattern, regex=True)]

# Display the comments containing URLs
comments_with_urls.head()

,clean_comment,category


In [69]:
# Identify comments containing new line characters
comments_with_newline = df[df['clean_comment'].str.contains('\n')]

# Display the comments containing new line characters
comments_with_newline.head()

,clean_comment,category
448,what missing jpg\nand why this brilliant edit ...,1
781,india has been ruined congress and populist sc...,-1
847,like aap for its stand corruption and making p...,-1
871,reduced trade\ndeficit stronger rupee aren the...,0
1354,amsa press conference australian maritime safe...,1


In [70]:
# Remove new line characters from the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].str.replace('\n', ' ', regex=True)

# Verify the transformation by checking for any remaining new lines
comments_with_newline_remaining = df[df['clean_comment'].str.contains('\n')]
comments_with_newline_remaining

,clean_comment,category


In [71]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [72]:
# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\b4bik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\b4bik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [75]:
df = pd.read_csv('Reddit_Data.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [77]:
df.dropna(inplace=True)

In [78]:
df.drop_duplicates(inplace=True)

In [79]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [80]:
# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [81]:
# Apply the preprocessing function to the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

In [82]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [95]:
def handle_negation(text):
    text = re.sub(r"not\s+(\w+)", r"not_\1", text)
    text = re.sub(r"never\s+(\w+)", r"never_\1", text)
    text = re.sub(r"no\s+(\w+)", r"no_\1", text)
    return text

df['clean_comment'] = df['clean_comment'].apply(handle_negation)

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [97]:
# Features and target
X = df['clean_comment']
y = df['category']   # <-- change if your label column name is different


In [98]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [89]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(
        max_features=15000,
        ngram_range=(1, 2),
        min_df=5
    )),
    ('model', RandomForestClassifier(
        n_estimators=300,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

pipeline.fit(X_train, y_train)

y_pred_pipe = pipeline.predict(X_test)

print("Pipeline Accuracy:", accuracy_score(y_test, y_pred_pipe))
print(classification_report(y_test, y_pred_pipe))


Pipeline Accuracy: 0.8043212392988178
              precision    recall  f1-score   support

          -1       0.83      0.49      0.62      1650
           0       0.80      0.95      0.87      2555
           1       0.80      0.85      0.83      3154

    accuracy                           0.80      7359
   macro avg       0.81      0.76      0.77      7359
weighted avg       0.81      0.80      0.79      7359



In [112]:
from sklearn.linear_model import LogisticRegression

pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=40000,
        ngram_range=(1, 3),
        min_df=3
    )),
    ('model', LogisticRegression(
        max_iter=300,
        class_weight='balanced',
        n_jobs=-1
    ))
])

pipeline_lr.fit(X_train, y_train)

y_pred_lr = pipeline_lr.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.830004076640848
              precision    recall  f1-score   support

          -1       0.76      0.74      0.75      1650
           0       0.81      0.92      0.86      2555
           1       0.89      0.80      0.85      3154

    accuracy                           0.83      7359
   macro avg       0.82      0.82      0.82      7359
weighted avg       0.83      0.83      0.83      7359



In [101]:
from sklearn.svm import LinearSVC

pipeline_svm = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=40000,
        ngram_range=(1, 3),
        min_df=3
    )),
    ('model', LinearSVC(class_weight='balanced'))
])

pipeline_svm.fit(X_train, y_train)

y_pred_svm = pipeline_svm.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


C:\Users\b4bik\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM Accuracy: 0.8490284005979073
              precision    recall  f1-score   support

          -1       0.80      0.73      0.76      1650
           0       0.83      0.93      0.88      2555
           1       0.89      0.84      0.87      3154

    accuracy                           0.85      7359
   macro avg       0.84      0.83      0.84      7359
weighted avg       0.85      0.85      0.85      7359



In [102]:
!pip install lightgbm


In [103]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report


In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=25000,
    ngram_range=(1, 3),
    min_df=5,
    max_df=0.9,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [114]:
lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=3,
    boosting_type='gbdt',
    learning_rate=0.05,
    n_estimators=500,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

lgb_model.fit(X_train_tfidf, y_train_enc)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.197677 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128581
[LightGBM] [Info] Number of data points in the train set: 29434, number of used features: 4334
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
               learning_rate=0.05, n_estimators=500, n_jobs=-1, num_class=3,
               num_leaves=64, objective='multiclass', random_state=42,
               subsample=0.8)

In [115]:
y_pred_enc = lgb_model.predict(X_test_tfidf)
y_pred = le.inverse_transform(y_pred_enc)

print("LightGBM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


LightGBM Accuracy: 0.8660144041309961
              precision    recall  f1-score   support

          -1       0.80      0.77      0.78      1650
           0       0.86      0.95      0.90      2555
           1       0.91      0.85      0.88      3154

    accuracy                           0.87      7359
   macro avg       0.86      0.86      0.85      7359
weighted avg       0.87      0.87      0.87      7359



In [116]:
lgb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=25000,
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        sublinear_tf=True
    )),
    ('model', lgb.LGBMClassifier(
        objective='multiclass',
        num_class=3,
        boosting_type='gbdt',
        learning_rate=0.05,
        n_estimators=500,
        max_depth=-1,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])


In [120]:
lgb_pipeline.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.160531 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 128581
[LightGBM] [Info] Number of data points in the train set: 29434, number of used features: 4334
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, max_features=25000, min_df=5,
                                 ngram_range=(1, 3), sublinear_tf=True)),
                ('model',
                 LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
                                learning_rate=0.05, n_estimators=500, n_jobs=-1,
                                num_class=3, num_leaves=64,
                                objective='multiclass', random_state=42,
                                subsample=0.8))])

In [121]:
y_pred = lgb_pipeline.predict(X_test)


print("LightGBM Pipeline Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


LightGBM Pipeline Accuracy: 0.8660144041309961
              precision    recall  f1-score   support

          -1       0.80      0.77      0.78      1650
           0       0.86      0.95      0.90      2555
           1       0.91      0.85      0.88      3154

    accuracy                           0.87      7359
   macro avg       0.86      0.86      0.85      7359
weighted avg       0.87      0.87      0.87      7359



In [126]:
lgb_pipeline.predict(['very good'])

array([1], dtype=int64)

In [127]:
import joblib

# Save pipeline
joblib.dump(lgb_pipeline, "lgb_model.pkl")

['lgb_model.pkl']